In [ ]:
# !chmod +x install_pyg_cpu.sh
# !./install_pyg_cpu.sh

In [5]:
import torch
import torch_geometric
import gemmi
import Bio
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader



In [8]:
import torch
from torch_geometric.loader import DataLoader

# Load preprocessed datasets
train_dataset = torch.load("protein_dna_dataset/train_dataset.pt")
val_dataset = torch.load("protein_dna_dataset/val_dataset.pt")
test_dataset = torch.load("protein_dna_dataset/test_dataset.pt")

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


/tmp/ipykernel_10225/3813946123.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load("protein_dna_dataset/train_dataset.pt")
/tmp/ipykernel_10225/3

In [9]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn as nn

class ProteinDNAGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(ProteinDNAGNN, self).__init__()

        # Graph convolutional layers
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)

        # Fully connected layer for regression
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))

        # Global pooling to get a graph-level representation
        x = global_mean_pool(x, batch)

        # Output the binding affinity prediction
        return self.fc(x)


In [10]:
# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = train_dataset[0].num_node_features  # Based on atomic features, e.g., atomic number
hidden_dim = 64  # You can experiment with this value

model = ProteinDNAGNN(input_dim, hidden_dim).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()  # Mean Squared Error for regression


In [11]:
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def validate():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in val_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y)
            total_loss += loss.item()
    return total_loss / len(val_loader)


In [12]:
epochs = 100  # You can increase this based on your dataset size

for epoch in range(1, epochs + 1):
    train_loss = train()
    val_loss = validate()
    
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "protein_dna_gnn_model.pth")


/home/marcus/miniconda3/envs/gnn-protein-dna/lib/python3.10/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/marcus/miniconda3/envs/gnn-protein-dna/lib/python3.10/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/marcus/miniconda3/envs/gnn-protein-dna/lib/python3.10/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to inco

Epoch 1: Train Loss = 62.2741, Val Loss = 66.0215
Epoch 10: Train Loss = 45.1733, Val Loss = 47.7606
Epoch 20: Train Loss = 13.1402, Val Loss = 13.4935


KeyboardInterrupt: 

In [13]:
def test():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y)
            total_loss += loss.item()
    return total_loss / len(test_loader)

test_loss = test()
print(f"Test Loss: {test_loss:.4f}")


Test Loss: 2.8280


/home/marcus/miniconda3/envs/gnn-protein-dna/lib/python3.10/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


# Make Predictions on New Data

In [ ]:
def predict(data):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        out = model(data.x, data.edge_index, data.batch)
    return out.item()

# Example prediction on a single protein-DNA complex
sample_data = test_dataset[0]  # Take the first test sample
predicted_affinity = predict(sample_data)
print(f"Predicted Binding Affinity: {predicted_affinity:.4f}")
